In [ ]:
#!/usr/bin/env python3
# coding: utf-8

from os import listdir
from numpy import asarray
from numpy import save
from glob import glob
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import sys
# import cv2 as cv
import os
import sys
from glob import glob
cwd = os.getcwd()

def replace_filename(input, pattern, replaceWith): 
    return input.replace(pattern, replaceWith) 

input_files = glob('*.jpg')
photos, labels = list(), list()
cat_files =[]
dog_files =[]

for i in range(len(input_files)):
    if 'Cat' in input_files[i]:
        cat_files.append(input_files[i])
        output = 1.0
        photo = load_img(input_files[i], target_size=(200, 200))
        photo.save(replace_filename(input_files[i],'Cat','resized_Cat'), 'JPEG', quality=90)
        photos.append(photo)
        labels.append(output)
    else:
        dog_files.append(input_files[i]) 
        output = 0.0
        photo = load_img(input_files[i], target_size=(200, 200))
        photo.save(replace_filename(input_files[i],'Dog','resized_Dog'), 'JPEG', quality=90)
        photos.append(photo)
        labels.append(output)  
    
with open('resized_images.txt', 'w') as f:
    for item in photos:
        f.write("%s\n" % item)

with open('labels.txt', 'w') as f:
    for item in labels:
        f.write("%s\n" % item)

In [16]:
#!/usr/bin/env python3
# coding: utf-8
import sys
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from optkeras.optkeras import OptKeras
import optkeras
import pickle
from keras.optimizers import RMSprop
import optuna
import os
import tensorflow as tf
import argparse
import joblib


    
def hpo_monitor(study, trial):
    with open(checkpoint_file, 'wb') as f:
        joblib.dump(study,checkpoint_file) 
    
def objective(trial):
    nb_classes = 2
    epochs=1
    batch_size =5
    optimizer_options = ["RMSprop", "Adam", "SGD"]

    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation=trial.suggest_categorical('activation', ['relu', 'linear']))(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)
    for layer in vgg16_model.layers:
        layer.trainable = False

    model.compile(optimizer = trial.suggest_categorical("optimizer", ["rmsprop", "Adam", "SGD"]),loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, 
              verbose=ok.keras_verbose ,validation_data=(val_photos,val_labels))
    return ok.trial_best_value


with open('training.pkl', 'rb') as f:
    train = pickle.load(f)

with open('testing.pkl', 'rb') as f:
     test = pickle.load(f)

with open('validation.pkl','rb') as f:
     val = pickle.load(f)

train_photos, train_labels = list(), list()
tp = list()
for file in train:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    train_photos.append(photo)
    train_labels.append(output)
train_photos = asarray(train_photos)
train_labels = asarray(train_labels)

test_photos, test_labels = list(), list()
for file in test:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    tp.append(photo)
    test_photos.append(photo)
    test_labels.append(output)
test_photos = asarray(test_photos)
test_labels = asarray(test_labels)

val_photos, val_labels = list(), list()
for file in val:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    val_photos.append(photo)
    val_labels.append(output)
val_photos = asarray(val_photos)
val_labels = asarray(val_labels)

train_photos =train_photos.astype('float32')
test_photos= test_photos.astype('float32')
val_photos =val_photos.astype('float32')
train_photos /= 255
val_photos /= 255
test_photos /= 255

optkeras.optkeras.get_trial_default = lambda: optuna.trial.FrozenTrial(
            None, None, None, None, None, None, None, None, None, None, None)
study_name = "CatsAndDogs" + '_Simple'
ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')

STUDY = None
N_TRIALS = 3

checkpoint_file = "study_checkpoint.pkl"

finished = 0
try:
    with open(checkpoint_file, 'rb') as f:
        STUDY = joblib.load(f)
    finished = len(STUDY.trials_dataframe())
            
    if finished < N_TRIALS:  
        todo_trials = N_TRIALS - finished
        if todo_trials > 0 :
            STUDY.optimize(objective, n_trials=todo_trials, timeout=600, callbacks=[hpo_monitor])
            
except Exception as e:
    STUDY = OptKeras(study_name=study_name,monitor='val_acc',direction='maximize')
    STUDY.optimize(objective, n_trials=N_TRIALS, timeout=600, callbacks=[hpo_monitor])



[I 2020-09-23 22:21:35,332] A new study created in memory with name: CatsAndDogs_Simple


[2020-09-23 22:21:35.337431] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)


[I 2020-09-23 22:21:35,342] A new study created in memory with name: CatsAndDogs_Simple


[2020-09-23 22:21:35.346967] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
[I 2020-09-23 22:21:43,650] Trial 0 finished with value: inf and parameters: {'activation': 'linear', 'optimizer': 'Adam'}. Best is trial 0 with value: inf.
[I 2020-09-23 22:21:51,540] Trial 1 finished with value: inf and parameters: {'activation': 'linear', 'optimizer': 'rmsprop'}. Best is trial 0 with value: inf.
[I 2020-09-23 22:21:59,446] Trial 2 finished with value: inf and parameters: {'activation': 'relu', 'optimizer': 'Adam'}. Best is trial 0 with value: inf.


[2020-09-23 22:21:59.478856] Trial#: 2, value: inf| Best trial#: 0, value: inf, params: {'activation': 'linear', 'optimizer': 'Adam'}


In [17]:
import joblib
with open('study_checkpoint.pkl', 'rb') as f:
    STUDY = joblib.load(f)

In [18]:
STUDY.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_activation,params_optimizer,state
0,0,inf,2020-09-23 22:21:35.351798,2020-09-23 22:21:43.650229,0 days 00:00:08.298431,linear,Adam,COMPLETE
1,1,inf,2020-09-23 22:21:43.663659,2020-09-23 22:21:51.540343,0 days 00:00:07.876684,linear,rmsprop,COMPLETE
2,2,inf,2020-09-23 22:21:51.552693,2020-09-23 22:21:59.445898,0 days 00:00:07.893205,relu,Adam,COMPLETE


In [24]:
#!/usr/bin/env python3
# coding: utf-8
import sys
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from optkeras.optkeras import OptKeras
import optkeras
import pickle
from keras.optimizers import RMSprop
import optuna
import os
import tensorflow as tf
import argparse
import joblib


with open('training.pkl', 'rb') as f:
    train = pickle.load(f)

with open('testing.pkl', 'rb') as f:
     test = pickle.load(f)

with open('validation.pkl','rb') as f:
     val = pickle.load(f)

train_photos, train_labels = list(), list()
tp = list()
for file in train:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    train_photos.append(photo)
    train_labels.append(output)
train_photos = asarray(train_photos)
train_labels = asarray(train_labels)

test_photos, test_labels = list(), list()
for file in test:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    tp.append(photo)
    test_photos.append(photo)
    test_labels.append(output)
test_photos = asarray(test_photos)
test_labels = asarray(test_labels)

val_photos, val_labels = list(), list()
for file in val:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    val_photos.append(photo)
    val_labels.append(output)
val_photos = asarray(val_photos)
val_labels = asarray(val_labels)

train_photos =train_photos.astype('float32')
test_photos= test_photos.astype('float32')
val_photos =val_photos.astype('float32')
train_photos /= 255
val_photos /= 255
test_photos /= 255

import os
from tensorflow import keras

# Prepare a directory to store all the checkpoints.
checkpoint_dir = './ckpt'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    
def make_model():
    # Create a model.
    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)

    for layer in vgg16_model.layers:
        layer.trainable = False

    model.compile(optimizer = 'rmsprop',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model

def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + '/' + name
                   for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print('Restoring from', latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print('Creating a new model')
    return make_model()

model = make_or_restore_model()
callbacks = [
    # This callback saves a SavedModel every 2 batches.
    # We include the training loss in the folder name.
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir + '/ckpt-loss={loss:.2f}')
]
model.fit(train_photos, epochs=2, callbacks=callbacks)
# model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=2, callbacks=callbacks,validation_data=(val_photos,val_labels))


Creating a new model


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:104: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


AttributeError: 'NoneType' object has no attribute 'shape'

In [25]:
#!/usr/bin/env python3
# coding: utf-8
import sys
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from optkeras.optkeras import OptKeras
import optkeras
import pickle
from keras.optimizers import RMSprop
import optuna
import os
import tensorflow as tf
import argparse
import joblib


    
def hpo_monitor(study, trial):
    with open('study_checkpoint.pkl', 'wb') as f:
        pickle.dumps(study, f)
#     joblib.dump(study,"study_checkpoint.pkl") 
    
def objective(trial):
    nb_classes = 2
    epochs=1
    batch_size =5
    optimizer_options = ["RMSprop", "Adam", "SGD"]

    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation=trial.suggest_categorical('activation', ['relu', 'linear']))(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)
    for layer in vgg16_model.layers:
        layer.trainable = False

    model.compile(optimizer = trial.suggest_categorical("optimizer", ["rmsprop", "Adam", "SGD"]),loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose ,validation_data=(val_photos,val_labels))
    return ok.trial_best_value


with open('training.pkl', 'rb') as f:
    train = pickle.load(f)

with open('testing.pkl', 'rb') as f:
     test = pickle.load(f)

with open('validation.pkl','rb') as f:
     val = pickle.load(f)

train_photos, train_labels = list(), list()
tp = list()
for file in train:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    train_photos.append(photo)
    train_labels.append(output)
train_photos = asarray(train_photos)
train_labels = asarray(train_labels)

test_photos, test_labels = list(), list()
for file in test:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    tp.append(photo)
    test_photos.append(photo)
    test_labels.append(output)
test_photos = asarray(test_photos)
test_labels = asarray(test_labels)

val_photos, val_labels = list(), list()
for file in val:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    val_photos.append(photo)
    val_labels.append(output)
val_photos = asarray(val_photos)
val_labels = asarray(val_labels)

train_photos =train_photos.astype('float32')
test_photos= test_photos.astype('float32')
val_photos =val_photos.astype('float32')
train_photos /= 255
val_photos /= 255
test_photos /= 255

optkeras.optkeras.get_trial_default = lambda: optuna.trial.FrozenTrial(
            None, None, None, None, None, None, None, None, None, None, None)
study_name = "CatsAndDogs" + '_Simple'
ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')


N_TRIALS = 2

checkpoint_file = "checkpoint.csv"
p = pd.read_csv(ok.optuna_log_file_path)
p.to_csv(checkpoint_file)


finished = 0
try:
    finished = len(pd.read_csv(checkpoint_file))
    if finished < N_TRIALS:  
        todo_trials = N_TRIALS - finished
        if todo_trials > 0 :
            ok.optimize(objective, n_trials=N_TRIALS, timeout=600)
            
except Exception as e:
    ok.optimize(objective, n_trials=N_TRIALS, timeout=600)

[I 2020-09-24 10:04:21,996] A new study created in memory with name: CatsAndDogs_Simple


[2020-09-24 10:04:22.001170] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


[2020-09-24 10:04:22.800752]  (None) 


[I 2020-09-24 10:04:31,376] Trial 0 finished with value: 0.8333333333333334 and parameters: {'activation': 'linear', 'optimizer': 'rmsprop'}. Best is trial 0 with value: 0.8333333333333334.


[2020-09-24 10:04:31.982495] Trial#: 0, value: 8.333333e-01| Best trial#: 0, value: 8.333333e-01, params: {'activation': 'linear', 'optimizer': 'rmsprop'}


[I 2020-09-24 10:04:40,411] Trial 1 finished with value: 0.5 and parameters: {'activation': 'relu', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.8333333333333334.


[2020-09-24 10:04:41.098734] Trial#: 1, value: 5.000000e-01| Best trial#: 0, value: 8.333333e-01, params: {'activation': 'linear', 'optimizer': 'rmsprop'}


[I 2020-09-24 10:04:50,149] Trial 2 finished with value: 0.5 and parameters: {'activation': 'linear', 'optimizer': 'Adam'}. Best is trial 0 with value: 0.8333333333333334.


[2020-09-24 10:04:50.178788] Trial#: 2, value: 5.000000e-01| Best trial#: 0, value: 8.333333e-01, params: {'activation': 'linear', 'optimizer': 'rmsprop'}


In [31]:
import pandas as pd
p = pd.read_csv(ok.optuna_log_file_path)
p

,n_u_m_b_e_r,v_a_l_u_e,d_a_t_e_t_i_m_e___s_t_a_r_t,d_a_t_e_t_i_m_e___c_o_m_p_l_e_t_e,d_u_r_a_t_i_o_n,p_a_r_a_m_s___a_c_t_i_v_a_t_i_o_n,p_a_r_a_m_s___o_p_t_i_m_i_z_e_r,u_s_e_r___a_t_t_r_s_____D_a_t_e_t_i_m_e___e_p_o_c_h___b_e_g_i_n,u_s_e_r___a_t_t_r_s_____D_a_t_e_t_i_m_e___e_p_o_c_h___e_n_d,u_s_e_r___a_t_t_r_s_____T_r_i_a_l___n_u_m,u_s_e_r___a_t_t_r_s___a_c_c,u_s_e_r___a_t_t_r_s___l_o_s_s,u_s_e_r___a_t_t_r_s___v_a_l___a_c_c,u_s_e_r___a_t_t_r_s___v_a_l___l_o_s_s,s_t_a_t_e
0,0,0.833333,2020-09-24 10:04:22.004920,2020-09-24 10:04:31.375714,0 days 00:00:09.370794,linear,rmsprop,2020-09-24 10:04:22.953468,2020-09-24 10:04:30.891147,0,0.547619,2.939940,0.833333,0.341130,COMPLETE
1,1,0.500000,2020-09-24 10:04:31.379090,2020-09-24 10:04:40.411406,0 days 00:00:09.032316,relu,SGD,2020-09-24 10:04:32.111070,2020-09-24 10:04:39.931260,1,0.476190,0.973174,0.500000,0.608491,COMPLETE
2,2,0.500000,2020-09-24 10:04:40.414605,2020-09-24 10:04:50.148724,0 days 00:00:09.734119,linear,Adam,2020-09-24 10:04:41.317571,2020-09-24 10:04:49.453194,2,0.428571,1.745774,0.500000,0.457280,COMPLETE


In [29]:
p.to_csv("checkpoint.csv")